In [26]:
import pandas as pd
import numpy as np
import os
import regex
from glob import glob
import warnings
warnings.simplefilter("ignore")

In [27]:
jenns_file = pd.read_excel("new_paper_1.xlsx")
master_list = pd.read_excel("Master-List-copy.xlsx")
gov_df = pd.read_excel('gov_side.xlsx', 'Full Publication List')

file_sep = os.path.sep
box_dir = file_sep + 'Users' + file_sep + os.getlogin() + file_sep + 'Box'
sero_net = box_dir + file_sep + 'SeroNet Public Data' 



In [28]:
jenns_file.head()
len(jenns_file.Base_Project)

15

In [29]:
def get_PI_name(jenn, master):
    pi_names = []
    grant_number = [e[3:] for e in jenn.Base_Project]
    for i in grant_number:
        name = glob(sero_net + file_sep + i + '*')[0].split('_')[1]
        matching = [s for s in np.unique(master.PI) if name.lower() in s.lower()]
        pi_names.append(matching[0])
    return(pi_names)


In [30]:
# finding new papers 
def new_paper_list(gov, master):
    new_papers_PMID = list(set(gov.PMID)- set(master.PMID))
    return(gov[gov['PMID'].isin(new_papers_PMID)])

In [31]:
#changing to correct format to merge to master list 
def to_master_format(new_papers, master):
    data_dic = {}
    empty = ['']*len(new_papers.PubMed_ID)
    updating_fields = {
        'Base_Project':list(new_papers['Base_Project']),
        'Base_Project.1':[e[3:] for e in new_papers.Base_Project], 
        'PMID':list(new_papers['PubMed_ID']), 
        'Folder name':list(['PMID_'+str(e) for e in new_papers.PubMed_ID]),
        'PubMed_ID link':list(new_papers['PubMed_ID']),
        'Publication_Title':list(new_papers['Publication_Title']), 
        'Journal':list(new_papers['Journal']),
        'Author(s)':list(new_papers['Author(s)']),
        'Pub Date':list(new_papers['Pub Date']),
        'PI':get_PI_name(new_papers, master_list)
    }

    for col_name in master_list.columns:
        if (col_name in updating_fields.keys()):
            data_dic[col_name] = updating_fields[col_name]
        else:
            data_dic[col_name] = empty

    return(pd.DataFrame(data_dic))

In [32]:
def find_repeat(list1, list2):
    return(set(list1).intersection(set(list2)))

In [33]:
# finding new papers 
new_papers = new_paper_list(gov_df, master_list)

# creating df to concat to master list
add_on = to_master_format(new_papers, master_list)
len(add_on.PMID)

0

In [34]:
new_papers

,Base_Project,PD_Name,PD_Email,PMID,PubMed_ID,Compliance_Status,Notification_Date,Journal,Publication_Title,Author(s),Pub Date,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16


In [35]:
new_master_list = pd.concat([master_list, add_on])
new_master_list = new_master_list.replace(np.nan, '', regex=True)
new_master_list.head()
new_master_list.columns

Index(['NOTES', 'Primary Research Topic', 'Secondary Research Topic',
       'Research Topic Comments', 'Publication Owner', 'JIRA Epic #',
       'Base_Project', 'Status/ImmPort Study ID', 'Base_Project.1', 'PMID',
       'Folder name', 'PubMed_ID link', 'Publication_Title', 'Journal',
       'Author(s)', 'Pub Date', 'PI', 'poc name', 'poc email', 'Status',
       'status comment', 'QA Review Responsible', 'QA review complete (Y/N)'],
      dtype='object')

In [36]:
new_master_list.head()

,NOTES,Primary Research Topic,Secondary Research Topic,Research Topic Comments,Publication Owner,JIRA Epic #,Base_Project,Status/ImmPort Study ID,Base_Project.1,PMID,...,Journal,Author(s),Pub Date,PI,poc name,poc email,Status,status comment,QA Review Responsible,QA review complete (Y/N)
0,SKIP (review),,,,,,U01CA260476,,CA260476,33581646.0,...,Current opinion in virology,"Webb, Nicholas E; Bernshtein, Biana; Alter, Galit",2021 Apr,"Alter, Galit",,,,,,
1,,Clinical,,,Liu,SPDC-125,U01CA260476,SDY1825,CA260476,33589636.0,...,Nature communications,"Bartsch, Yannic C; Fischinger, Stephanie; Sidd...",2021 02 15,"Alter, Galit",gentle reminder sent 8/16,,pending POC,,,
2,,Clinical,,,Liu,SPDC-129,U01CA260476,SDY1787,CA260476,33589825.0,...,Nature medicine,"Yannic C Bartsch # 1, Chuangqi Wang # 2, Tomer...",,"Alter, Galit",gentle reminder sent 8/16,,pending POC,,,
3,,Clinical,,,Liu,SPDC-161,U01CA260476,SDY1826,CA260476,33775692.0,...,American journal of obstetrics and gynecology,"Gray, Kathryn J; Bordt, Evan A; Atyeo, Carolin...",2021 Mar 26,"Alter, Galit",gentle reminder sent 8/16,,pending POC,,,
4,,Clinical,,,Liu,SPDC-61,U01CA260476,SDY1817,CA260476,33207184.0,...,Cell,"Zohar, Tomer; Loos, Carolin; Fischinger, Steph...",2020 12 10,"Alter, Galit",gentle reminder sent 8/16,,pending POC,,CC,Y


In [227]:
a = list(set(gov_df.PubMed_ID)- set(master_list.PMID))
b = list(set(jenns_file.PubMed_ID))  
set(a) - set(b)

set()

In [19]:
a.sort() == b.sort()

True